In [464]:
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_curve, roc_auc_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import History 
import xgboost as xgb
from sklearn.metrics import accuracy_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


What factors decide the grades of students? What combinations of them? Can we predict them?

In [441]:
mat = pd.read_csv("../input/student-alcohol-consumption/student-mat.csv")
por = pd.read_csv("../input/student-alcohol-consumption/student-por.csv")
#matpor = pd.merge(mat,por, on=("school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"))
matANDpor = pd.concat([mat,por]).drop_duplicates(subset=("school","sex","age","address","famsize","Pstatus","Medu","Fedu","Mjob","Fjob","reason","nursery","internet"))

Lets plot the grade distributions for all binary variables to see if there is any resolving potential there

In [442]:
# Visualise grade distributions based on the binary discriminants

def plots(subject, period):
    df = subject
    subject_list = []
    counter = -1
    for i in df.keys():
        if len(pd.unique(df[[i]].values.ravel('K'))) == 2:
            subject_list.append(i)
    half = int(np.ceil(len(subject_list)/2))
    fig, ax = plt.subplots(half,2)
    fig.set_size_inches(15, 25)
    for a in subject_list:
        counter += 1
        if counter < half:
            j = 0
            k = 0
        else:
            j = 1
            k = half
        df1 = df[df[a] == pd.unique(df[[a]].values.ravel('K'))[0]]
        df2 = df[df[a] == pd.unique(df[[a]].values.ravel('K'))[1]]
        ax[counter-k][j].hist(df1[period], bins=20, alpha=0.5, label=pd.unique(df[[a]].values.ravel('K'))[0])
        ax[counter-k][j].hist(df2[period], bins=20, alpha=0.5, label=pd.unique(df[[a]].values.ravel('K'))[1])
        ax[counter-k][j].set_title(a)
        ax[counter-k][j].legend()

plots(matANDpor, "G3")

# We can see some skew in some distributions as expected, although the relative sizes of data are
# often very different

We can take two routes from here - see if we there is any correlation between grades in different periods, or try to predict a grade in general based on the other variables. Lets take the second route.

In [443]:
# Separate grades and cobine into one column
grades = ["G1", "G2", "G3"]
df_list = []
for i in range(len(grades)):
    df = matANDpor.copy()
    a = 0
    while a < len(grades):
        if a != i:
            df.drop(grades[a], axis=1, inplace=True)
        a+=1
    df.rename(columns={grades[i] : "G"}, inplace=True)
    df_list.append(df)
df = pd.concat(df_list)

In [444]:
# Prepare "bad" and "good" grade scores (less than 11 and more than 10) for binary classification

x_0 = df[(df["G"]<11)]
x_1 = df[(df["G"]>10)]
y_0 = pd.DataFrame(np.zeros(x_0.shape[0]))
y_1 = pd.DataFrame(np.ones(x_1.shape[0]))

y = pd.concat([y_0,y_1])
y.columns = ["class"]
x = pd.concat([x_0,x_3])

In [445]:
# Replace binary string options to 0 or 1

def binarize(df):
    for i in df.keys():
        if len(pd.unique(df[[i]].values.ravel('K'))) == 2:
            df[i].replace({pd.unique(df[[i]].values.ravel('K'))[1] : 1,pd.unique(df[[i]].values.ravel('K'))[0] : 0},inplace=True)

binarize(x)

In [446]:
# Drop nominal columns - these are harder to deal with and theres only four of them. Drop the grades 
# as well

x = x.drop(["Mjob","Fjob", "reason", "guardian", "G"], axis=1).reset_index(drop=True) 

# recale variables so that they go between 0-1 
scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
print(scaler_x.fit(x))
xscale=scaler_x.transform(x)

x = pd.DataFrame(xscale,columns=x.columns)

In [447]:
# split X1, X2, and y into train and validation dataset 

x_train, x_test, y_train, y_test  = train_test_split(
    x,
    y,
    test_size=0.2,
    random_state=123456,
    stratify=y.values,
)

In [448]:
# define a simple NN
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(len(x.columns), input_dim=len(x.columns), kernel_initializer='normal', activation='relu'))
    model.add(Dense((len(x.columns))*2, kernel_initializer='normal', activation='relu'))
    model.add(Dense(1, activation="sigmoid"))
    model.compile(loss='binary_crossentropy', optimizer='adam')#, metrics=['accuracy'])  
    return model

# define early stopping
early_stop = EarlyStopping(monitor='val_loss',patience=10)

In [449]:
history = History()

model = baseline_model()

model.fit(x_train, y_train,
          #sample_weight=w_train,
          batch_size=100,
          epochs=100,
          callbacks=[history,early_stop],
          validation_data=(x_test, y_test))#, w_val))

In [450]:
# Extract number of run epochs from the training history
epochs = range(1, len(history.history["loss"])+1)

# Extract loss on training and validation ddataset and plot them together
plt.plot(epochs, history.history["loss"], "o-", label="Training")
plt.plot(epochs, history.history["val_loss"], "o-", label="Test")
plt.xlabel("Epochs"), plt.ylabel("Loss")
plt.yscale("log")
plt.legend();

In [451]:
prediction = model.predict(x_test)

In [452]:
#  define a function to plot the ROC curves - just makes the roc_curve look nicer than the default
def plot_roc_curve(fpr, tpr, auc):
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr)
    ax.set(xlabel='False Positive Rate', ylabel='True Positive Rate')
    ax.grid()
    ax.text(0.6, 0.3, 'ROC AUC Score: {:.3f}'.format(auc),
            bbox=dict(boxstyle='square,pad=0.3', fc='white', ec='k'))
    lims = [np.min([ax.get_xlim(), ax.get_ylim()]), np.max([ax.get_xlim(), ax.get_ylim()])]
    ax.plot(lims, lims, 'k--')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    plt.savefig('roc_rho_rho_NN')

In [453]:
# plot ROC curve for improved training
y_proba = model.predict(x_test) # outputs two probabilties
auc = roc_auc_score(y_test, y_proba)
fpr, tpr, _ = roc_curve(y_test, y_proba)
plot_roc_curve(fpr, tpr, auc)

This is good predicting power, but lets try our luck with a BDT instead. Lets use more categories.

In [454]:
x_0 = df[(df["G"]<11)]
x_1 = df[(df["G"]>10)]
y_0 = pd.DataFrame(np.zeros(x_0.shape[0]))
y_1 = pd.DataFrame(np.ones(x_1.shape[0]))

y = pd.concat([y_0,y_1])
y.columns = ["class"]
x = pd.concat([x_0,x_1])

In [455]:
def binarize(df):
    for i in df.keys():
        if len(pd.unique(df[[i]].values.ravel('K'))) == 2:
            df[i].replace({pd.unique(df[[i]].values.ravel('K'))[1] : 1,pd.unique(df[[i]].values.ravel('K'))[0] : 0},inplace=True)

binarize(x)

x = x.drop(["Mjob","Fjob", "reason", "guardian", "G"], axis=1).reset_index(drop=True) 

scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
print(scaler_x.fit(x))
xscale=scaler_x.transform(x)

x = pd.DataFrame(xscale,columns=x.columns)

In [456]:
x_train, x_test, y_train, y_test  = train_test_split(
    x,
    y,
    test_size=0.2,
    random_state=123456,
    stratify=y.values,
)

In [457]:
# define some XGBoost parameters, unspecified will be default
# https://xgboost.readthedocs.io/en/latest////index.html
# not optimised at all, just playing by ear

xgb_params = {
    "objective": "binary:logistic",
    "max_depth": 5,
    "learning_rate": 0.02,
    "silent": 1,
    "n_estimators": 1000,
    "subsample": 0.9,
    "seed": 123451,
}

In [458]:
xgb_clf = xgb.XGBClassifier(**xgb_params)
xgb_clf.fit(
    x_train,
    y_train,
    early_stopping_rounds=200, 
    eval_set=[(x_train, y_train), (x_test, y_test)],
    eval_metric = "auc",
    verbose=True,
)

In [459]:
# look at feature importance
# can use different metrics (weight or gain)
xgb.plot_importance(xgb_clf, importance_type='weight')
xgb.plot_importance(xgb_clf, importance_type='gain')

Weight ranks the variables to which the BDT are used the most often, whereas the gain shows which are the most indicative. Clearly failures and absences matter a lot.

In [460]:
def plot_roc_curve(fpr, tpr, auc):
    fig, ax = plt.subplots()
    ax.plot(fpr, tpr)
    ax.set(xlabel='False Positive Rate', ylabel='True Positive Rate')
    ax.grid()
    ax.text(0.6, 0.3, 'ROC AUC Score: {:.3f}'.format(auc),
            bbox=dict(boxstyle='square,pad=0.3', fc='white', ec='k'))
    lims = [np.min([ax.get_xlim(), ax.get_ylim()]), np.max([ax.get_xlim(), ax.get_ylim()])]
    ax.plot(lims, lims, 'k--')
    ax.set_xlim(lims)
    ax.set_ylim(lims)
    plt.savefig('roc_rho_rho')

In [462]:
y_proba = xgb_clf.predict_proba(x_test)
auc = roc_auc_score(y_test, y_proba[:,1])
fpr, tpr, _ = roc_curve(y_test, y_proba[:,1])
plot_roc_curve(fpr, tpr, auc)

In [466]:
y_proba = xgb_clf.predict_proba(x_test)
idx = y_proba.argmax(axis=1)
y_pred = (idx[:,None] == np.arange(y_proba.shape[1])).astype(float)
flatpred = np.argmax(y_pred, axis=-1)
flattest = np.argmax(y_test, axis=-1)
flattest = flattest["class"].tolist()
#print(accuracy_score(y_test, y_pred), " Convolutional Model")

In [467]:
#~~ Creating confusion arrays ~~#
truelabels = np.array([[0,0],[0,0]]) #for true modes 0,1,2,3
lengthstrue = [0,0]
lengthspred = [0,0]
for a in range(len(flattest)):
    truelabels[int(flattest[a])][int(flatpred[a])] +=1
    lengthstrue[int(flattest[a])] +=1
    lengthspred[int(flatpred[a])] +=1
truelabelpurity = truelabels/lengthspred
truelabelefficiency = np.array([[0,0],[0,0]], dtype = float)
for a in range(2):
    for b in range(2):
        truelabelefficiency[a][b] = truelabels[a][b]/lengthstrue[a]

In [468]:
#~~ PLOTTING CONFUSION MATRICES ~~#
plt.rcParams.update({'figure.autolayout': True})
labellist = ['Bad', 'Good']
fig, ax = plt.subplots(1,2)
plt.tight_layout()
fig.set_size_inches(12, 8)

ax[0].imshow(truelabelefficiency, cmap = 'Blues')
for i in range(2):
    for j in range(2):
        if truelabelefficiency[i, j] > 0.5:
            text = ax[0].text(j, i, round(truelabelefficiency[i, j], 3),
                           ha="center", va="center", color="w")
        else:
            text = ax[0].text(j, i, round(truelabelefficiency[i, j], 3),
                           ha="center", va="center", color="black")

        
ax[0].set_title('Efficiency')
ax[0].set_xticks([0,1])
ax[0].set_yticks([0,1])
ax[0].set_xticklabels(labellist)
ax[0].set_yticklabels(labellist)
ax[0].set_xlabel('Predicted Mode')
ax[0].set_ylabel('True Mode')


ax[1].imshow(truelabelpurity, cmap = 'Blues')
for i in range(2):
    for j in range(2):
        if truelabelpurity[i, j] > 0.5:
            text = ax[1].text(j, i, round(truelabelpurity[i, j], 3),
                           ha="center", va="center", color="w")
        else:
            text = ax[1].text(j, i, round(truelabelpurity[i, j], 3),
                           ha="center", va="center", color="black")

ax[1].set_title('Purity')
ax[1].set_xticks([0,1])
ax[1].set_yticks([0,1])
ax[1].set_xticklabels(labellist)
ax[1].set_yticklabels(labellist)
ax[1].set_xlabel('Predicted Grade')
ax[1].set_ylabel('True Grade')

Now lets try with mor categories.

In [469]:
x_0 = df[(df["G"]<11)]
x_1 = df[(df["G"]>4) & (df["G"]<11)]
x_2 = df[(df["G"]>10) & (df["G"]<16)]
x_3 = df[(df["G"]>10)]
y_0 = pd.DataFrame(np.zeros(x_0.shape[0]))
y_1 = pd.DataFrame(np.ones(x_1.shape[0]))
y_2 = pd.DataFrame(2*np.ones(x_2.shape[0]))
y_3 = pd.DataFrame(3*np.ones(x_3.shape[0]))

y = pd.concat([y_0,y_1,y_2,y_3])
y.columns = ["class"]
x = pd.concat([x_0,x_1,x_2,x_3])

def binarize(df):
    for i in df.keys():
        if len(pd.unique(df[[i]].values.ravel('K'))) == 2:
            df[i].replace({pd.unique(df[[i]].values.ravel('K'))[1] : 1,pd.unique(df[[i]].values.ravel('K'))[0] : 0},inplace=True)

binarize(x)

x = x.drop(["Mjob","Fjob", "reason", "guardian", "G"], axis=1).reset_index(drop=True) 

scaler_x = MinMaxScaler()
scaler_y = MinMaxScaler()
print(scaler_x.fit(x))
xscale=scaler_x.transform(x)

x = pd.DataFrame(xscale,columns=x.columns)

x_train, x_test, y_train, y_test  = train_test_split(
    x,
    y,
    test_size=0.2,
    random_state=123456,
    stratify=y.values,
)

xgb_params = {
    "objective": "multi:softprob",
    "max_depth": 5,
    "learning_rate": 0.02,
    "silent": 1,
    "n_estimators": 1000,
    "subsample": 0.9,
    "seed": 123451,
}

xgb_clf = xgb.XGBClassifier(**xgb_params)
xgb_clf.fit(
    x_train,
    y_train,
    early_stopping_rounds=200, 
    eval_set=[(x_train, y_train), (x_test, y_test)],
    eval_metric = "auc",
    verbose=True,
)

In [470]:
xgb.plot_importance(xgb_clf, importance_type='weight')
xgb.plot_importance(xgb_clf, importance_type='gain')

In [475]:
y_proba = xgb_clf.predict_proba(x_test)
auc = roc_auc_score(y_test, y_proba[:,1], multi_class="ovr",average=None)
fpr, tpr, _ = roc_curve(y_test, y_proba[:,1])
plot_roc_curve(fpr, tpr, auc)

In [472]:
y_proba = xgb_clf.predict_proba(x_test)
idx = y_proba.argmax(axis=1)
y_pred = (idx[:,None] == np.arange(y_proba.shape[1])).astype(float)
flatpred = np.argmax(y_pred, axis=-1)
flattest = np.argmax(y_test, axis=-1)
flattest = flattest["class"].tolist()
print(accuracy_score(y_test, y_pred), " Convolutional Model")

In [473]:
#~~ Creating confusion arrays ~~#
truelabels = np.array([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]) #for true modes 0,1,2,3
lengthstrue = [0,0,0,0]
lengthspred = [0,0,0,0]
for a in range(len(flattest)):
    truelabels[int(flattest[a])][int(flatpred[a])] +=1
    lengthstrue[int(flattest[a])] +=1
    lengthspred[int(flatpred[a])] +=1
truelabelpurity = truelabels/lengthspred
truelabelefficiency = np.array([[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]], dtype = float)
for a in range(4):
    for b in range(4):
        truelabelefficiency[a][b] = truelabels[a][b]/lengthstrue[a]

In [474]:
plt.rcParams.update({'figure.autolayout': True})
labellist = ['Very Bad', 'Bad', 'Good', 'Very Good']
fig, ax = plt.subplots(1,2)
plt.tight_layout()
fig.set_size_inches(12, 8)

ax[0].imshow(truelabelefficiency, cmap = 'Blues')
for i in range(4):
    for j in range(4):
        if truelabelefficiency[i, j] > 0.5:
            text = ax[0].text(j, i, round(truelabelefficiency[i, j], 3),
                           ha="center", va="center", color="w")
        else:
            text = ax[0].text(j, i, round(truelabelefficiency[i, j], 3),
                           ha="center", va="center", color="black")

        
ax[0].set_title('Efficiency')
ax[0].set_xticks([0,1,2,3])
ax[0].set_yticks([0,1,2,3])
ax[0].set_xticklabels(labellist)
ax[0].set_yticklabels(labellist)
ax[0].set_xlabel('Predicted Mode')
ax[0].set_ylabel('True Mode')


ax[1].imshow(truelabelpurity, cmap = 'Blues')
for i in range(4):
    for j in range(4):
        if truelabelpurity[i, j] > 0.5:
            text = ax[1].text(j, i, round(truelabelpurity[i, j], 3),
                           ha="center", va="center", color="w")
        else:
            text = ax[1].text(j, i, round(truelabelpurity[i, j], 3),
                           ha="center", va="center", color="black")

ax[1].set_title('Purity')
ax[1].set_xticks([0,1,2,3])
ax[1].set_yticks([0,1,2,3])
ax[1].set_xticklabels(labellist)
ax[1].set_yticklabels(labellist)
ax[1].set_xlabel('Predicted Grade')
ax[1].set_ylabel('True Grade')